# Vector DB Test

벡터디비 활용을 통해 임베딩 데이터 저장 및 검색 테스트<br>
>임베딩 추출 시간 절약, 데이터 저장 및 검색 효율성 증대 목적

## milvus test

오픈소스 벡터디비 milvus의 경량화 버전 milvus-lite 구동 테스트<br>
> 파이썬 연결 지원, 파일 기반 데이터 제어, 로컬 환경 구축

In [1]:
import fiftyone as fo

fo.list_datasets()

['noise_composite_current',
 'noise_composite_merged',
 'noise_composite_pred',
 'tunelpdr_merged_all']

In [10]:
dataset = fo.load_dataset("noise_composite_merged")
sample = dataset.first()

sample.get_field_schema()

OrderedDict([('id', <fiftyone.core.fields.ObjectIdField at 0x7fa72722e300>),
             ('filepath',
              <fiftyone.core.fields.StringField at 0x7fa727a4d460>),
             ('tags', <fiftyone.core.fields.ListField at 0x7fa72722f6e0>),
             ('metadata',
              <fiftyone.core.fields.EmbeddedDocumentField at 0x7fa736979490>),
             ('created_at',
              <fiftyone.core.fields.DateTimeField at 0x7fa72722f4a0>),
             ('last_modified_at',
              <fiftyone.core.fields.DateTimeField at 0x7fa766bab1a0>),
             ('ground_truth',
              <fiftyone.core.fields.EmbeddedDocumentField at 0x7fa7297b4d40>),
             ('source', <fiftyone.core.fields.StringField at 0x7fa72910e930>),
             ('clip_embeddings',
              <fiftyone.core.fields.ListField at 0x7fa727b7bb60>)])

In [28]:
print(sample.id)
print(len(sample.id))
print(type(sample.id))

67c91348df3013a998789607
24
<class 'str'>


In [23]:
from pymilvus import MilvusClient
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection

db_client = MilvusClient("../db/DAE_data.db")

schema = db_client.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)
schema.add_field(field_name="sample_id", datatype=DataType.VARCHAR, is_primary=True, max_length=24)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=512)

db_client.create_collection(
    collection_name="test_collection",
    schema=schema
)
res = db_client.get_load_state(collection_name="test_collection")
print(res)

{'state': <LoadState: Loaded>}


In [26]:
type(sample)

fiftyone.core.sample.Sample

In [ ]:
data